# Flyer for tomography scans

Configure the Flyers for tomo scans.  Support taxi & fly.

In [1]:
import ophyd
ophyd.setup_ophyd()

import bluesky
from bluesky.utils import get_history
RE = bluesky.RunEngine(get_history())

from bluesky.plan_tools import print_summary
from bluesky.simulators import summarize_plan
import bluesky.plan_stubs as bps
import bluesky.plans as bp
import databroker

import numpy as np
import bluesky.magics
from datetime import datetime
import time

# Uncomment the following lines to turn on 
# verbose messages for debugging.
# import logging
# ophyd.logger.setLevel(logging.DEBUG)
# logging.basicConfig(level=logging.DEBUG)

# diagnostics
#RE.msg_hook = bluesky.utils.ts_msg_hook

db = databroker.Broker.named("mongodb_config")
RE.subscribe(db.insert)

RE.preprocessors.append(bluesky.SupplementalData())

from bluesky.magics import BlueskyMagics
get_ipython().register_magics(BlueskyMagics)

#from bluesky.callbacks.broker import verify_files_saved
#RE.subscribe(post_run(verify_files_saved), 'stop')

import APS_BlueSky_tools.devices

Loading metadata history from /home/beams/USER2BMB/.config/bluesky/bluesky_history.db


In [2]:
aps_current = ophyd.EpicsSignalRO("S:SRcurrentAI", name="aps_current")

try:
    A_shutter = APS_BlueSky_tools.devices.ApsPssShutter("2bma:A_shutter", name="A_shutter")
    B_shutter = APS_BlueSky_tools.devices.ApsPssShutter("2bma:B_shutter", name="B_shutter")
except Exception as _exc:
    print(_exc)

In [3]:
from enum import Enum


class BusyStatus(str, Enum):
    busy = "Busy"
    done = "Done"

In [ ]:
# TODO: modify for 2-bm-b
BUSY_PV = '2bmb:PSOFly:taxi'
#BUSY_PV = '2bmb:PSOFly:fly'

class BusyFlyerDeviceBase(ophyd.Device):
    """
    support a fly scan that is triggered by a busy record
    """

    # TODO: define in the constructor instead using PV as parameter
    busy = ophyd.Component(ophyd.EpicsSignal, "", string=True)
    
    def __init__(self, *args, **kwargs):
        super().__init__('', parent=None, **kwargs)
        self.complete_status = None

    def kickoff(self):
        """
        Start this Flyer
        """
        logger.info("kickoff()")
        self.complete_status = DeviceStatus(self.busy)
        
        def cb(*args, **kwargs):
            if self.busy.value in (BusyStatus.done):
                self.complete_status._finished(success=True)
        
        self.busy.put(BusyStatus.busy)
        self.busy.subscribe(cb)

        kickoff_status = DeviceStatus(self)
        kickoff_status._finished(success=True)
        return kickoff_status

    def complete(self):
        """
        Wait for flying to be complete
        """
        logger.info("complete(): " + str(self.complete_status))
        return self.complete_status

    def describe_collect(self):
        """
        Describe details for ``collect()`` method
        """
        logger.info("describe_collect()")
        schema = {}
        # TODO: What will be returned?
        return {self.name: schema}

    def collect(self):
        """
        Start this Flyer
        """
        logger.info("collect(): " + str(self.complete_status))
        self.complete_status = None
        # TODO: What will be yielded?


class BusyFlyerDeviceTaxi(BusyFlyerDeviceBase):
    busy = ophyd.Component(ophyd.EpicsSignal, "2bmb:PSOFly1:taxi", string=True)


class BusyFlyerDeviceFly(BusyFlyerDeviceBase):
    busy = ophyd.Component(ophyd.EpicsSignal, "2bmb:PSOFly1:fly", string=True)


taxi_flyer = BusyFlyerDeviceTaxi(name="taxi_flyer")
fly_flyer = BusyFlyerDeviceFly(name="fly_flyer")


In [ ]:
taxi_flyer.busy.value